In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
import os

In [2]:
CS_file = '/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/REAL_samples/deconprepare/corr/allmethodsummary/try4_corrplot/7pbmc_firstflow_m8real2.txt'
est_files = ['/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/REAL_samples/deconprepare/corr/allmethodsummary/try4_corrplot/BL14_genepromSM_0.7.txt_bestref.txt_result_dupindex_binnedstats.pkladjustedScore_minus_avg_CSxOut.txt']

pearson_file = 'correlation_CS/pearson_comCD8.txt'
spearman_file = 'correlation_CS/spearman_comCD8.txt'

p_data = pd.read_csv(pearson_file, sep='\t', index_col=0)
s_data = pd.read_csv(spearman_file, sep='\t', index_col=0)

df_p = pd.DataFrame(p_data)
df_s = pd.DataFrame(s_data)

CS_data = pd.read_csv(CS_file, index_col=0)
df_CS = pd.DataFrame(CS_data)
df_CS = df_CS.sort_index()

outfol = '/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/REAL_samples/deconprepare/corr/allmethodsummary/try4_corrplot/firtgt'
os.mkdir(outfol)

FileNotFoundError: [Errno 2] File correlation_CS/pearson_comCD8.txt does not exist: 'correlation_CS/pearson_comCD8.txt'

In [ ]:
df_CS.columns

In [ ]:
est_cells = ['CD4', 'CD8', 'NK', 'Mn', 'mNeu', 'nB']
CS_cells = ['T cells CD4 naive', 'T cells CD8', 'NK cells resting/NK cells activated', 'Monocytes', 'Neutrophils', 'B cells naive']

In [ ]:
def get_cell_vals(cell, df):
    split = cell.split('/')
    val = np.zeros(len(np.array(df[split[0]])))
    for i in range(len(split)):
        val += np.array(df[split[i]])
    return val

In [ ]:
for j in range(len(est_files)):
    fig, axs = plt.subplots(1, len(est_cells), figsize=(10 * len(est_cells), 10))
    est_data = pd.read_csv(est_files[j], sep='\t', index_col=0)
    df_est = pd.DataFrame(est_data)
    df_est = df_est.sort_index()
    for i in range(len(est_cells)):
        est_arr = get_cell_vals(est_cells[i], df_est)
        CS_arr = get_cell_vals(CS_cells[i], df_CS)
        m, b = np.polyfit(CS_arr, est_arr, 1)
        axs[i].scatter(CS_arr, est_arr)
        axs[i].plot(CS_arr, m*CS_arr + b, '--', alpha=0.5)
        axs[i].set_title(est_cells[i])
        axs[i].set_xlabel('CIBERSORTx')
        axs[i].set_ylabel('Estimation (' + est_files[j].split('_')[-4] + ')')
        corr_col = ''
        for col in list(df_p.columns):
            if est_cells[i] in col:
                corr_col = col
        p_corr = np.around(float(df_p[(df_p.index == est_files[j].split('/')[1])][corr_col]), decimals=3)
        s_corr = np.around(float(df_s[(df_s.index == est_files[j].split('/')[1])][corr_col]), decimals=3)
        xy = (0.1, 0.1)
        axs[i].text(0.1, 0.9, 'r = ' + str(p_corr) + '\nrho = ' + str(s_corr), horizontalalignment='left', verticalalignment='bottom', transform=axs[i].transAxes)
    
    fig.suptitle(est_files[0])
    plt.savefig(outfol + '/' + est_files[j].split('/')[1] + '_figure_separate_cell.pdf', dpi=300, bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1, len(est_files), figsize=(10 * len(est_files), 10))

for j in range(len(est_files)):
    est_data = pd.read_csv(est_files[j], sep='\t', index_col=0)
    df_est = pd.DataFrame(est_data)
    df_est = df_est.sort_index()
    diag_arr = np.array([])
    est_tot_arr = np.array([])
    p_arr = np.zeros(len(est_cells))
    s_arr = np.zeros(len(est_cells))
    for i in range(len(est_cells)):
        est_arr = get_cell_vals(est_cells[i], df_est)
        CS_arr = get_cell_vals(CS_cells[i], df_CS)
        m, b = np.polyfit(CS_arr, est_arr, 1)
        axs[j].scatter(CS_arr, est_arr, label=est_cells[i])
        diag_arr = np.concatenate((diag_arr, CS_arr), axis=None)
        est_tot_arr = np.concatenate((est_tot_arr, est_arr))
        corr_col = ''
        for col in list(df_p.columns):
            if est_cells[i] in col:
                corr_col = col
        p_corr = float(df_p[(df_p.index == est_files[j].split('/')[1])][corr_col])
        s_corr = float(df_s[(df_s.index == est_files[j].split('/')[1])][corr_col])
        p_arr[i] = p_corr
        s_arr[i] = s_corr
    p_corr_mean = np.around(np.mean(p_corr), decimals=3)
    s_corr_mean = np.around(np.mean(s_corr), decimals=3)
    m, b = np.polyfit(diag_arr, est_tot_arr, 1)
    axs[j].plot(diag_arr, m*diag_arr + b, '--', alpha=0.3, c='k')
    axs[j].text(0.1, 0.9, 'r = ' + str(p_corr_mean) + '\nrho = ' + str(s_corr_mean), horizontalalignment='left', verticalalignment='bottom', transform=axs[j].transAxes)
    axs[j].set_title(est_files[j].split('_')[-4])
    axs[j].set_xlabel('CIBERSORTx')
    axs[j].set_ylabel('Estimation')
    plt.legend(loc=(1.05,0.72), fontsize=16)
    
plt.savefig(outfol + '/' + est_files[j].split('/')[1] + '_figure_combined_cell.pdf', dpi=300, bbox_inches='tight')